In [0]:
%sql
SHOW TABLES;

In [0]:
%sql
SELECT * from silver;

In [0]:
%sql

-- Set the table format to Delta (Databricks standard)
CREATE OR REPLACE TABLE gold_inflation_index
USING DELTA 
AS
-- The Gold Layer Calculation: Quarterly Price Index and Inflation Rate
WITH Quarterly_Avg AS (
    SELECT
        commodity_category,
        country_code,
        DATE_TRUNC('quarter', date) AS quarter_start_date,
        AVG(COALESCE(price_per_kg, price_per_litre)) AS avg_quarterly_price
    FROM silver 
    WHERE price_per_kg IS NOT NULL OR price_per_litre IS NOT NULL 
    GROUP BY 1, 2, 3
),
Inflation_Calculations AS (
    SELECT
        *,
        LAG(avg_quarterly_price, 1) OVER (
            PARTITION BY commodity_category, country_code
            ORDER BY quarter_start_date
        ) AS prev_quarter_price
    FROM Quarterly_Avg
)
-- Final Output: Calculate the Inflation Rate with rounding
SELECT
    commodity_category,
    country_code,
    DATE_FORMAT(quarter_start_date, 'yyyy-MM-dd') AS quarter_start_date,
    ROUND(avg_quarterly_price, 4) AS avg_quarterly_price_usd,
    ROUND(prev_quarter_price, 4) AS prev_quarter_price_usd,
    ROUND((avg_quarterly_price - prev_quarter_price)*100 / prev_quarter_price, 4) AS qoq_inflation_rate
FROM Inflation_Calculations
WHERE prev_quarter_price IS NOT NULL
ORDER BY country_code, quarter_start_date DESC;

In [0]:
%sql
select commodity_category,ROUND(avg(qoq_inflation_rate)*4, 2) as average_inflation from gold_inflation_index
group by commodity_category;

In [0]:
%sql
select * from gold_inflation_index
where commodity_category='Sugar';

In [0]:
%sql
--We take a closer look at the outliers
SELECT
    MIN(qoq_inflation_rate) AS min_qoq_rate,
    MAX(qoq_inflation_rate) AS max_qoq_rate,
    COUNT(*) AS total_quarters
FROM gold_inflation_index
WHERE commodity_category = 'Sugar';


In [0]:
%sql

-- Create or replace the final, clean Gold layer table (Delta format) without the outliers
CREATE OR REPLACE TABLE cleaned_gold_inflation
USING DELTA
AS
    --  Filter out the confirmed extreme outliers (Sugar spike)
    SELECT
        commodity_category,
        country_code,
        quarter_start_date,
        avg_quarterly_price_usd,
        qoq_inflation_rate
    FROM gold_inflation_index -- Use the existing Gold Table with all QoQ data
    -- Filter out anything over 500% QoQ 
    WHERE qoq_inflation_rate BETWEEN -100 AND 500



In [0]:
%sql
select commodity_category,ROUND(avg(qoq_inflation_rate)*4, 2) as average_inflation from 
cleaned_gold_inflation
group by commodity_category;

In [0]:
%sql
select country_code,ROUND(avg(qoq_inflation_rate)*4, 2) as average_inflation from 
cleaned_gold_inflation
group by country_code
order by average_inflation desc;

In [0]:
%sql
select quarter_start_date,ROUND(avg(qoq_inflation_rate), 2) as average_inflation from 
cleaned_gold_inflation
group by 1
order by 1 desc;